In [1]:
import sys
import os
import gc
import logging
sys.path.append(r"..")
from utils import *
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix

from collections import defaultdict
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.client import device_lib
import logging
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(
    filename='my.log', 
    level=logging.DEBUG, 
    format=LOG_FORMAT)

path_build = "./build/"
path_embed = "./embed/"
path_list = "./feature_series/"
path_mnt = "./mnt/"
path_sub = "./sub/"
print(tf.__version__)
#print(tf.test.is_built_with_gpu_support)
#print(tf.test.is_gpu_available())
#print(device_lib.list_local_devices())
#os.environ["CUDA_VISIBLE_DEVICES"] = '0' #use GPU with ID=0
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
#对需要进行限制的GPU进行设置
# tf.config.experimental.set_virtual_device_configuration(gpus[0],
#                                                       [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
# gpus

2.1.0
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [4]:
creative_id_em = load_pickle(f"{path_embed}/creative_id_w2v_matrix.pkl")
#ad_id_em = load_pickle(f"{path_embed}/ad_id_w2v_matrix.pkl")
advertiser_id_em = load_pickle(f"{path_embed}/advertiser_id_w2v_matrix.pkl")
product_id_em = load_pickle(f"{path_embed}/product_id_w2v_matrix.pkl")
industry_em = load_pickle(f"{path_embed}/industry_w2v_matrix.pkl")
product_category_em = load_pickle(f"{path_embed}/product_category_w2v_matrix.pkl")
# click_times_em = load_pickle(f"{path_embed}/click_times_w2v_matrix2.pkl")
# time_em = load_pickle(f"{path_embed}/time_w2v_matrix2.pkl")

#creative_id_times_em = load_pickle(f"{path_embed}/creative_id_times_w2v_matrix.pkl")
#ad_id_times_em = load_pickle(f"{path_embed}/ad_id_times_w2v_matrix3.pkl")
#product_id_times_em = load_pickle(f"{path_embed}/product_id_times_w2v_matrix.pkl")
#advertiser_id_times_em = load_pickle(f"{path_embed}/advertiser_id_times_w2v_matrix.pkl")
#product_category_times_em = load_pickle(f"{path_embed}/product_category_times_w2v_matrix.pkl")
#industry_times_em = load_pickle(f"{path_embed}/industry_times_w2v_matrix.pkl")

# creative_id_t_em = load_pickle(f"{path_save}/creative_id_t_w2v_matrix2.pkl")
# # ad_id_t_em = load_pickle(f"{path_save}/ad_id_t_w2v_matrix2.pkl")
# product_id_t_em = load_pickle(f"{path_save}/product_id_t_w2v_matrix2.pkl")
# advertiser_id_t_em = load_pickle(f"{path_save}/advertiser_id_t_w2v_matrix2.pkl")
# product_category_t_em = load_pickle(f"{path_save}/product_category_t_w2v_matrix2.pkl")
# industry_t_em = load_pickle(f"{path_save}/industry_t_w2v_matrix2.pkl")

#time_clicktimes_em = load_pickle(f"{path_embed}/time_clicktimes_w2v_matrix2.pkl")
#time_creativeids_em = load_pickle(f"{path_embed}/time_creativeids_w2v_matrix2.pkl")

w2v_features = [
    {'name':'creative_id', 'size':256, 'windows':10, 'min_count':1, 'version':1, 'max_len':128, 'em':creative_id_em},
    #{'name':'ad_id', 'size':128, 'windows':5, 'min_count':1, 'version':1, 'max_len':128, 'em':ad_id_em},
    {'name':'advertiser_id', 'size':64, 'windows':10, 'min_count':1, 'version':1, 'max_len':128, 'em':advertiser_id_em},
    {'name':'product_id', 'size':64, 'windows':10, 'min_count':1, 'version':1, 'max_len':128, 'em':product_id_em},
    {'name':'industry', 'size':32, 'windows':10, 'min_count':1, 'version':1, 'max_len':128, 'em':industry_em},
    {'name':'product_category', 'size':16, 'windows':10, 'min_count':1, 'version':1, 'max_len':128, 'em':product_category_em},
#     {'name':'time', 'size':16, 'windows':5, 'min_count':1, 'version':1, 'max_len':128, 'em':time_em},
#     {'name':'click_times', 'size':8, 'windows':5, 'min_count':1, 'version':1, 'max_len':128, 'em':click_times_em},
    
    #{'name':'creative_id_times', 'size':256, 'windows':10, 'min_count':1, 'version':1, 'max_len':128, 'em':creative_id_times_em},
    #{'name': 'ad_id_times', 'size': 128, 'windows': 10, 'min_count': 1, 'version': 1, 'max_len':128, 'em':ad_id_times_em},
    #{'name':'product_id_times', 'size':64, 'windows':10, 'min_count':1, 'version':1, 'max_len':128, 'em':product_id_times_em},
    #{'name':'advertiser_id_times', 'size':64, 'windows':10, 'min_count':1, 'version':1, 'max_len':128, 'em':advertiser_id_times_em},
    #{'name':'product_category_times', 'size':32, 'windows':10, 'min_count':1, 'version':1,'max_len':128, 'em':product_category_times_em},
    #{'name':'industry_times', 'size':32, 'windows':10, 'min_count':1, 'version':1,'max_len':128, 'em':industry_times_em},   
    
    
    #{'name':'time_clicktimes', 'size':91, 'windows':10, 'min_count':1, 'version':2, 'vocab_size':5000,'max_len':128,'em':time_clicktimes_em},
    #{'name':'time_creativeids', 'size':91, 'windows':10, 'min_count':1, 'version':2,'vocab_size':5000, 'max_len':128,'em': time_creativeids_em},
    
]
dense_features = ['creative_id_len', 'ad_id_len', 'product_id_len', 'product_category_len', 'advertiser_len', 'industry_len','time_len',
                  'mean_clicktimes', 'max_clicktimes', 'min_clicktimes', 'mean_time', 'max_time', 'min_time']  
dense_features = []
base_features = ['product_id', 'product_category', 'advertiser_id', 'industry']
#base_features = ['creative_id', 'ad_id']
for fea in base_features:
    for g in [1]:
        dense_features.append(f'mean_prob_{fea}_gender_{g}')
        dense_features.append(f'max_prob_{fea}_gender_{g}')
        dense_features.append(f'min_prob_{fea}_gender_{g}')
        dense_features.append(f'var_prob_{fea}_gender_{g}')
    for a in [1,2,3,4,5,6,7,8,9,10]:
        dense_features.append(f'mean_prob_{fea}_age_{a}')
        dense_features.append(f'max_prob_{fea}_age_{a}')
        dense_features.append(f'min_prob_{fea}_age_{a}')
        dense_features.append(f'var_prob_{fea}_age_{a}')
dense_features = []

#print(creative_id_em.shape,ad_id_em.shape,advertiser_id_em.shape,product_id_em.shape)
#print(industry_em.shape,product_category_em.shape,click_times_em.shape,time_em.shape)

In [22]:
def lstm_model(units,num_classes,w2v_features, dense_features):
    inputs_dict = dict()
    embed_layer_list = []
    click_times_input = keras.Input(shape=(128,), name='click_times')
    inputs_dict['click_times'] = click_times_input
    
    click_times_input_exp = tf.expand_dims(click_times_input, axis=2)
    print(click_times_input_exp)
    for w2v_f in w2v_features:
        em_name = w2v_f['name']
        if em_name == 'click_times':continue
        em_size = w2v_f['em'].shape[0]
        em_dim = w2v_f['size']
        em_m = w2v_f['em']
        max_len = w2v_f['max_len']
        
        print(em_name)    
        inputs = keras.Input(shape=(max_len,), name=em_name)
        inputs_dict[em_name] = inputs
        embed_layer_list.append(
            keras.layers.multiply([keras.layers.Embedding(em_size, em_dim, input_length=max_len, trainable=False, weights=[em_m],mask_zero=True)(inputs),
                                   tf.tile(click_times_input_exp, [1,1,em_dim])]))
#         embed_layer_list.append( keras.layers.Embedding(
#                 em_size, em_dim, input_length=max_len, trainable=False, weights=[em_m],mask_zero=True)(inputs))
    embed_output = keras.layers.concatenate(embed_layer_list, axis=-1)
    #embed_output = keras.layers.Conv1D(512, 5, padding='same', kernel_initializer='normal', activation='relu')(embed_output)
    
    lstm_output0 = keras.layers.Bidirectional(keras.layers.LSTM(units,return_sequences=True))(embed_output)
    
    lstm_output = keras.layers.Bidirectional(keras.layers.LSTM(units,return_sequences=True))(lstm_output0)
    lstm_output = keras.layers.Bidirectional(keras.layers.LSTM(units,return_sequences=True))(lstm_output)
    lstm_output = keras.layers.Bidirectional(keras.layers.LSTM(units,return_sequences=True))(lstm_output)
    lstm_output = keras.layers.Bidirectional(keras.layers.LSTM(units,return_sequences=True))(lstm_output)
    lstm_output = keras.layers.Add()([lstm_output,lstm_output0])
    #lstm_output = layers.GlobalMaxPooling1D()(lstm_output)
                                     
    
    lstm_output = layers.concatenate([layers.GlobalAveragePooling1D()(lstm_output),
                                      layers.GlobalMaxPooling1D()(lstm_output),
                                     ], axis=-1)
    #lstm_output = layers.BatchNormalization()(lstm_output)
    lstm_output = layers.Dropout(0.3)(lstm_output)
    
    fc = keras.layers.Dense(units, activation='relu')(lstm_output)
    #数值型特征
    numeric_list = []
    for den_f in dense_features:
        inputs = keras.Input(shape=(1,), name=den_f)
        inputs_dict[den_f] = inputs
        numeric_list.append(inputs)   
    if dense_features != []:
        numeric_output = keras.layers.concatenate(numeric_list, axis=-1)
        numeric_output_bn = layers.BatchNormalization()(numeric_output)
        lstm_numeric_output = keras.layers.concatenate([fc,numeric_output_bn], axis=-1)
    else:
        lstm_numeric_output = fc
    
    
    outputs = keras.layers.Dense(num_classes, activation='softmax')(lstm_numeric_output)
    

        
    model = keras.Model(inputs=inputs_dict, outputs=outputs)
    model.compile(optimizer = keras.optimizers.Adam(0.001),
              loss = keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])
    return model

In [3]:
# def input_fn(feature_dict, label=None, epochs=5, shuffle=True, batch_size=64, fit_key='train'):
#     if fit_key == 'train':
#         dataset = tf.data.Dataset.from_tensor_slices((feature_dict, label))
#     else:
#         dataset = tf.data.Dataset.from_tensor_slices((feature_dict))
#     if shuffle:
#         dataset = dataset.shuffle(100*batch_size)
#     dataset = dataset.repeat(epochs).batch(batch_size)
    
#     return dataset

def input_fn(feature_dict, label=None, epochs=5, shuffle=True, batch_size=64, fit_key='train'):
    if fit_key == 'train':
        dataset = tf.data.Dataset.from_tensor_slices((feature_dict, label))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((feature_dict))
    if shuffle:
        dataset = dataset.shuffle(100*batch_size)
    dataset = dataset.batch(batch_size)
    
    return dataset


# def meta_dict_gen_train():
#     for i in range(2700000):
#         ls = {}
#         for key, val in feature_dict.items():
#             ls[key] = val[i]
#         yield ls
    
    
# def input_fn(feature_dict, label=None, epochs=5, shuffle=True, batch_size=64, fit_key='train'):
#     if fit_key == 'train':
#         dataset = tf.data.Dataset.from_generator(
#             meta_dict_gen_train,
#             output_types=({k: tf.float32 for k in feature_dict}, tf.int32),
#         )
#     else:
#         dataset = tf.data.Dataset.from_generator(
#             meta_dict_gen_train,
#             output_types=({k: tf.float32 for k in feature_dict},),
#         )
#     if shuffle:
#         dataset = dataset.shuffle(100*batch_size)
#     dataset = dataset.repeat(epochs).batch(batch_size)
    
#     return dataset



# ems = dict()
# with tf.device("/CPU:0"):
#     for w in w2v_features:
#         ems[w['name']] = tf.constant(w['em'])
# def data_embedding_lookup(X):
#     for f in X:
#         X[f] = tf.nn.embedding_lookup(ems[f],X[f])
#     return X

# def input_fn(feature_dict, label=None, epochs=5, shuffle=True, batch_size=64, fit_key='train'):
#     if fit_key == 'train':
#         dataset = tf.data.Dataset.from_tensor_slices((feature_dict, label)).map(lambda X,y:(data_embedding_lookup(X),y), num_parallel_calls=8)
#     else:
#         dataset = tf.data.Dataset.from_tensor_slices((feature_dict)).map(lambda X:(data_embedding_lookup(X)), num_parallel_calls=8)
#     #print(dataset)
#     if shuffle:
#         dataset = dataset.shuffle(100*batch_size)
#     dataset = dataset.repeat(epochs).batch(batch_size)
    
#     return dataset.map(map_func=lambda x,y:(x,y), num_parallel_calls=8)



In [6]:
users = pd.read_pickle(f"{path_build}/train_user.pkl")
users.sort_values(by=['user_id'], ascending=[True], inplace=True)
users = users.reset_index(drop=True)

users_mnt = pd.read_pickle(f"{path_mnt}/user_mnts.pkl")
w2v_features.append({'name':'click_times', 'max_len':128}) #添加click_times
fold_train = False
train_split = [0,2700000]
val_split = [2700000, 3000000]
test_split = [3000000]
train_feature_dict = dict()
val_feature_dict = dict()
test_feature_dict = dict()
if fold_train:
    kfolder = KFold(n_splits=5, shuffle=True, random_state=2020)
    kfold = kfolder.split(user_ids[0:3000000])
    fold_index = 2
    FLOD_TRAIN = 1
    for train_index, vali_index in kfold:
        if fold_index <= FLOD_TRAIN:
            fold_index += 1
            continue
        print(train_index, vali_index)
        break
    
    for fea in w2v_features:
        name = fea['name']
        max_length = fea['max_len']
        print(name)
        user_ids = np.load(f"{path_list}{name}_list_int.npy", allow_pickle=True)        
        train_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
            user_ids[train_index],value = 0,padding = 'post',maxlen = max_length )
        val_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
            user_ids[vali_index],value = 0,padding = 'post',maxlen = max_length )
        test_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
            user_ids[test_split[0]:],value = 0,padding = 'post',maxlen = max_length )
    for fea in dense_features:
        print(fea)
        train_feature_dict[fea] = users_mnt[fea][train_index]
        val_feature_dict[fea] = users_mnt[fea][vali_index]
        test_feature_dict[fea] = users_mnt[fea][test_split[0]:]

    gender_train_label = np.array(users['gender'][train_index])
    gender_val_label = np.array(users['gender'][vali_index])

    age_train_label = np.array(users['age'][train_index])
    age_val_label = np.array(users['age'][vali_index])
    
else:
    for fea in w2v_features:
        name = fea['name']
        max_length = fea['max_len']
        print(name)
        user_ids = np.load(f"{path_list}{name}_list_int.npy", allow_pickle=True)
        train_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
            user_ids[train_split[0]:train_split[1]],value = 0,padding = 'post',maxlen = max_length )
        val_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
            user_ids[val_split[0]:val_split[1]],value = 0,padding = 'post',maxlen = max_length )
        test_feature_dict[name] = keras.preprocessing.sequence.pad_sequences(
            user_ids[test_split[0]:],value = 0,padding = 'post',maxlen = max_length )
        
    for fea in dense_features:
        print(fea)
        train_feature_dict[fea] = users_mnt[fea][train_split[0]:train_split[1]]
        val_feature_dict[fea] = users_mnt[fea][val_split[0]:val_split[1]]
        test_feature_dict[fea] = users_mnt[fea][test_split[0]:]

    gender_train_label = np.array(users['gender'][train_split[0]:train_split[1]])
    gender_val_label = np.array(users['gender'][val_split[0]:val_split[1]])

    age_train_label = np.array(users['age'][train_split[0]:train_split[1]])
    age_val_label = np.array(users['age'][val_split[0]:val_split[1]])

creative_id
advertiser_id
product_id
industry
product_category
click_times


In [ ]:
def train_gender():
    num_classes = 2
    units = 128
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        gender_model = lstm_model(units, num_classes, w2v_features, dense_features)
    train_dataset = input_fn(train_feature_dict, gender_train_label-1, epochs=5, shuffle=True, batch_size=512)
    val_dataset = input_fn(val_feature_dict, gender_val_label-1, epochs=1, shuffle=False, batch_size=1024)
    early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0,
                                  patience=3, verbose=0, mode='auto',
                                  baseline=None, restore_best_weights=True)
    gender_model.fit(train_dataset, validation_data=val_dataset, epochs=100, callbacks = [early_stopping])

    test_dataset = input_fn(test_feature_dict, epochs=1, shuffle=False, batch_size=1024, fit_key='predict')
    gender_prob = gender_model.predict(test_dataset)
    gender_val_prob = gender_model.predict(val_dataset)
    print(gender_prob.shape,gender_val_prob.shape)
    tune_weight = search_weight(gender_val_label-1, gender_val_prob, init_weight=[1.0]*2,class_num=2, step=0.001)

    gender_prob_tune = np.array(tune_weight)*gender_prob
    gender_pre = np.argmax(gender_prob_tune,axis=1) + 1
    np.save(f"{path_sub}/val_gender_prob.npy", gender_val_prob)
    np.save(f"{path_sub}/gender_prob.npy", gender_prob)
from multiprocessing import Process
p = Process(target=train_gender)
p.start()
p.join() # 进程结束后，GPU显存会自动释放

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [23]:

num_classes = 10
units = 128
#mirrored_strategy = tf.distribute.MirroredStrategy()
# #devices=["/gpu:0", "/gpu:1"]
#with mirrored_strategy.scope():
age_model = lstm_model(units, num_classes, w2v_features, dense_features)
    
    
    
# train_dataset = input_fn(train_feature_dict, age_train_label-1, epochs=6, shuffle=True, batch_size=512)
# val_dataset = input_fn(val_feature_dict, age_val_label-1, epochs=1, shuffle=False, batch_size=1024)

# early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0,
#                               patience=3, verbose=0, mode='auto',
#                               baseline=None, restore_best_weights=True)

# age_model.fit(train_dataset, validation_data=val_dataset,epochs=100, callbacks = [early_stopping])

# test_dataset = input_fn(test_feature_dict, epochs=1, shuffle=False, batch_size=1024, fit_key='predict')
# age_prob = age_model.predict(test_dataset)
# age_val_prob = age_model.predict(val_dataset)
# print(age_prob.shape,age_val_prob.shape)
# age_tune_weight = search_weight(age_val_label-1, age_val_prob, init_weight=[1.0]*10,class_num=10, step=0.001)
# print(age_tune_weight)

# age_prob_tune = np.array(age_tune_weight)*age_prob
# age_pre = np.argmax(age_prob_tune,axis=1) + 1


# np.save(f"{path_sub}/val_age_prob.npy", age_val_prob)
# np.save(f"{path_sub}/age_prob.npy", age_prob)
age_model.summary()

Tensor("ExpandDims_1:0", shape=(None, 128, 1), dtype=float32)
creative_id
advertiser_id
product_id
industry
product_category
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
click_times (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
creative_id (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_op_layer_ExpandDims_1 (Tenso [(None, 128, 1)]     0           click_times[0][0]                
__________________________________________________________________________________________________
advertiser_id (InputLayer)      [(None, 128)]        0            

In [ ]:
print(gender_pre)
sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = age_pre
sub['predicted_gender'] = gender_pre
print('ok！')
sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')

In [ ]:
# for fea in w2v_features:
#     name = fea['name']
#     max_length = fea['max_len']
#     print(name)
#     user_ids = np.load(f"{path_list}{name}_list.npy", allow_pickle=True)
#     res = pd.Series(user_ids).map(lambda x:[int(i) for i in x])
#     np.save(f"{path_list}{name}_list_int.npy", res)

In [12]:
num_classes = 10
units = 128
age_model = lstm_model(units, num_classes, w2v_features, dense_features)

In [13]:
age_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
creative_id (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
advertiser_id (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
product_id (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
industry (InputLayer)           [(None, 128)]        0                                            
______________________________________________________________________________________________